# How to call functions with chat models

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`functions` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

If the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `function_call` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `function_call` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_call"` in the response, as well as a `function_call` object that has the name of the function and the generated function arguments.

### Overview

This notebook contains the following 2 sections:

- **How to generate function arguments:** Specify a set of functions and use the API to generate function arguments.
- **How to call functions with model generated arguments:** Close the loop by actually executing functions with model generated arguments.

## How to generate function arguments

In [66]:
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor 
!pip install openai
!pip install requests

In [67]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"

In [68]:
from getpass import getpass

openai.api_key = getpass("OpenAI Key: ")

### Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [69]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [70]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### Basic concepts

Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [71]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
    {
        "name": "get_n_day_weather_forecast",
        "description": "Get an N-day weather forecast",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
                "num_days": {
                    "type": "integer",
                    "description": "The number of days to forecast",
                }
            },
            "required": ["location", "format", "num_days"]
        },
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [72]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': 'Sure, I can help you with that. Can you please provide me with the city and state?'}

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [80]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}'}}

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'Game',
  'arguments': '{\n  "players": [\n    { "player_name": "Alice" },\n    { "player_name": "Bob" }\n  ],\n  "moves": [],\n  "location": "Glasgow, Scotland"\n}'}}

# Not adhering to a JSON schema (with `minItems`)

In [74]:
from pydantic import BaseModel, Field


class Player(BaseModel):
    """A player."""
    player_name: str

class Move(BaseModel):
    """A move."""
    player: Player
    move: str

class Game(BaseModel):
    """A game."""
    players: list[Player] = Field(description="List of players")
    moves: list[Move] = Field(description="List of moves", min_items=1)

game: Game = Game(**{
    "players": [
        {"player_name": "Alice"},
        {"player_name": "Bob"},
    ],
    "moves": [
        {"player": {"player_name": "Alice"}, "move": "move 1 to 2"},
        {"player": {"player_name": "Bob"}, "move": "move 2 to 3"},
    ],
})
game

Game(players=[Player(player_name='Alice'), Player(player_name='Bob')], moves=[Move(player=Player(player_name='Alice'), move='move 1 to 2'), Move(player=Player(player_name='Bob'), move='move 2 to 3')])

In [75]:
Game.model_json_schema()

{'$defs': {'Move': {'description': 'A move.',
   'properties': {'player': {'$ref': '#/$defs/Player'},
    'move': {'title': 'Move', 'type': 'string'}},
   'required': ['player', 'move'],
   'title': 'Move',
   'type': 'object'},
  'Player': {'description': 'A player.',
   'properties': {'player_name': {'title': 'Player Name', 'type': 'string'}},
   'required': ['player_name'],
   'title': 'Player',
   'type': 'object'}},
 'description': 'A game.',
 'properties': {'players': {'description': 'List of players',
   'items': {'$ref': '#/$defs/Player'},
   'title': 'Players',
   'type': 'array'},
  'moves': {'description': 'List of moves',
   'items': {'$ref': '#/$defs/Move'},
   'minItems': 1,
   'title': 'Moves',
   'type': 'array'}},
 'required': ['players', 'moves'],
 'title': 'Game',
 'type': 'object'}

In [76]:
# https://stackoverflow.com/a/58938747
def remove_a_key(d, remove_key):
    if isinstance(d, dict):
        for key in list(d.keys()):
            if key == remove_key:
                del d[key]
            else:
                remove_a_key(d[key], remove_key)


def schema_to_function(schema: ...):
    assert schema.__doc__, f"{schema.__name__} is missing a docstring."
    schema_dict = schema.model_json_schema()
    remove_a_key(schema_dict, "title")

    return {
        "name": schema.__name__,
        "description": schema.__doc__,
        "parameters": schema_dict,
    }


schema_to_function(Game)

{'name': 'Game',
 'description': 'A game.',
 'parameters': {'$defs': {'Move': {'description': 'A move.',
    'properties': {'player': {'$ref': '#/$defs/Player'},
     'move': {'type': 'string'}},
    'required': ['player', 'move'],
    'type': 'object'},
   'Player': {'description': 'A player.',
    'properties': {'player_name': {'type': 'string'}},
    'required': ['player_name'],
    'type': 'object'}},
  'description': 'A game.',
  'properties': {'players': {'description': 'List of players',
    'items': {'$ref': '#/$defs/Player'},
    'type': 'array'},
   'moves': {'description': 'List of moves',
    'items': {'$ref': '#/$defs/Move'},
    'minItems': 1,
    'type': 'array'}},
  'required': ['players', 'moves'],
  'type': 'object'}}

In [77]:
messages = []
messages.append({"role": "system", "content": "Create a chess game."})
messages.append({"role": "user", "content": "The players are Alice and Bob."})
chat_response = chat_completion_request(
    messages, functions=[schema_to_function(Game)], function_call={"name": "Game"}
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'Game',
  'arguments': '{\n  "players": [\n    { "player_name": "Alice" },\n    { "player_name": "Bob" }\n  ],\n  "moves": []\n}'}}

In [78]:
print(assistant_message["content"])

None


In [79]:
print(assistant_message["function_call"]["arguments"])

{
  "players": [
    { "player_name": "Alice" },
    { "player_name": "Bob" }
  ],
  "moves": []
}


In [84]:
import json

Game(**json.loads(assistant_message["function_call"]["arguments"]))

ValidationError: 1 validation error for Game
moves
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.1.2/v/too_short